In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/NLP_Project/twitter_data/
!ls

/content/drive/MyDrive/NLP_Project/twitter_data
agr_en_fb_gold.csv  tc_a.csv  tc_b.csv	tw_fb_gold.csv	val.csv  val_d.csv


In [25]:
!pip install chart_studio, nltk

ERROR: Invalid requirement: 'chart_studio,'


In [17]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
# from bs4 import BeautifulSoup
# import plotly.graph_objs as go
# from chart_studio import plotly
# import cufflinks
# from IPython.core.interactiveshell import InteractiveShell
# import plotly.figure_factory as ff
# InteractiveShell.ast_node_interactivity = 'all'
# from plotly.offline import iplot
# cufflinks.go_offline()
# cufflinks.set_config_file(world_readable=True, theme='pearl')

In [4]:
df = pd.read_csv('tw_fb_gold.csv')

In [5]:
def print_plot(index):
    example = df[df.index == index][['Tweet', 'Category']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Category:', example[1])

print_plot(10)

India is a fake, Fragile and unatural union build by British to control the poor masses. it has become a monster for minorities and neighbors, almost 40% of Indian Land is direct under Indian Army control who commit grave human right abuses under notorious AFSPA , Kashmiris hate India for brutal illegal occupation of their land but shameless India so called upper Class Hindu elite doesnt want to Free Kashmir for their political gaines, India is the only country which is being run by Extremists apart from Israel.
Category: 0


## Text Preprocessing

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:

df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text


In [8]:
df['Tweet'] = df['Tweet'].apply(clean_text)
df['Tweet'] = df['Tweet'].str.replace('\d+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [9]:
print_plot(10)

india fake fragile unatural union build british control poor masses become monster minorities neighbors almost  indian land direct indian army control commit grave human right abuses notorious afspa kashmiris hate india brutal illegal occupation land shameless india called upper class hindu elite doesnt want free kashmir political gaines india country run etremists apart israel
Category: 0


## LSTM Modelling

In [10]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 3000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 200
# This is fixed.
EMBEDDING_DIM = 70

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Tweet'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 10094 unique tokens.


In [11]:
# Truncate and pad the input sequences so that they are all in the same length for modeling
X = tokenizer.texts_to_sequences(df['Tweet'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2105, 200)


In [12]:
print(type(X))
print(len(X))
y = pd.get_dummies(df['Category']).values
print(type(y))
print(len(y))

<class 'numpy.ndarray'>
2105
<class 'numpy.ndarray'>
2105


In [13]:
X_train, X_val, y_train, y_val= train_test_split(X,y, test_size = 0.20, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)

(1684, 200) (1684, 3)
(421, 200) (421, 3)


In [14]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [15]:
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [16]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [21]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(20, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, precision_m, recall_m])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 70)           210000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 200, 70)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional_1 (Bidirectio  (None, 40)               14560     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 3)                 123       
                                                                 
Total params: 224,683
Trainable params: 224,683
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
epochs = 5
batch_size = 32

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
48/48 [==============================] - 85s 2s/step - loss: 1.0936 - accuracy: 0.3743 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.0824 - val_accuracy: 0.4793 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/5
48/48 [==============================] - 75s 2s/step - loss: 1.0261 - accuracy: 0.5512 - f1_m: 0.0262 - precision_m: 0.2160 - recall_m: 0.0143 - val_loss: 1.0208 - val_accuracy: 0.6036 - val_f1_m: 0.0202 - val_precision_m: 0.3333 - val_recall_m: 0.0104
Epoch 3/5
48/48 [==============================] - 75s 2s/step - loss: 0.8151 - accuracy: 0.7241 - f1_m: 0.4896 - precision_m: 0.8660 - recall_m: 0.3668 - val_loss: 0.9281 - val_accuracy: 0.5740 - val_f1_m: 0.5248 - val_precision_m: 0.7029 - val_recall_m: 0.4207
Epoch 4/5
48/48 [==============================] - 75s 2s/step - loss: 0.5428 - accuracy: 0.8205 - f1_m: 0.7906 - precision_m: 0.8782 - recall_m: 0.7221 - val_loss: 0.9354 - val_accuracy:

In [23]:
accr = model.evaluate(X_val,y_val)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

14/14 [==============================] - 1s 102ms/step - loss: 0.9563 - accuracy: 0.6081 - f1_m: 0.5927 - precision_m: 0.6329 - recall_m: 0.5585
Test set
  Loss: 0.956
  Accuracy: 0.608


## Seeing Predictions

In [24]:
before_tweets = pd.read_csv('tc_b.csv')
after_tweets = pd.read_csv('tc_a.csv')

In [25]:
before_labels = list(before_tweets['Category'])
after_labels = list(after_tweets['Category'])

before_texts = list(before_tweets['Tweet'])
# print([before_texts[10]])
seq_b = tokenizer.texts_to_sequences([before_texts[56]])
padded_b = pad_sequences(seq_b, maxlen=MAX_SEQUENCE_LENGTH)


after_texts = list(after_tweets['Tweet'])
seq_a = tokenizer.texts_to_sequences([after_texts[56]])
padded_a = pad_sequences(seq_a, maxlen=MAX_SEQUENCE_LENGTH)

In [26]:

pred_b = model.predict(padded_b)
pred_a = model.predict(padded_a)
labels = [0, 1, 2]

labels_dt = {0:'NAG', 1:'CAG', 2:'OAG'}

print('A before election tweet')
print(before_texts[56], labels_dt[labels[np.argmax(pred_b)]])

print('An after electin tweet')
print(after_texts[56], labels_dt[labels[np.argmax(pred_a)]])

A before election tweet
 KCR is absolutey right. No incumbent Prime Minister had such an empty campaign. He literally has no policy record on which he can seek votes all he has is more of the same: Hindu-Muslim Nehru ‘main toh fakeer hoon’ etc NAG
An after electin tweet
 Big developing story. @RahulGandhi adamant on resignation as @INCIndia looks for a successor. Just the shock the party needs to wake up from decades-old lazy complacency CAG
